In [131]:
import pickle
import re
import statistics as st

# Анализ словарей

## Словарь dic_a

In [77]:
# Посмотрим на первый словарь
with open('93.a.cache.pkl', 'rb') as pkl_file:
    dic_a = pickle.load(pkl_file)

dic_a

{'407ETR_FC01_20140402_092945_00020.jpg': [{'plate': 'BSYP875',
   'confidence': 94.22808074951172,
   'matches_template': 1,
   'plate_index': 0,
   'region': 'ca-on',
   'region_confidence': 86,
   'processing_time_ms': 24.62602424621582,
   'requested_topn': 20,
   'coordinates': [{'x': 1263, 'y': 254},
    {'x': 1398, 'y': 258},
    {'x': 1396, 'y': 310},
    {'x': 1259, 'y': 306}],
   'vehicle_region': {'x': 1100, 'y': 0, 'width': 458, 'height': 458},
   'candidates': [{'plate': 'BSYP875',
     'confidence': 94.22808074951172,
     'matches_template': 1}]}],
 '407ETR_FC01_20140402_093341_00021.jpg': [{'plate': '746APN',
   'confidence': 78.55723571777344,
   'matches_template': 1,
   'plate_index': 0,
   'region': 'ca-on',
   'region_confidence': 99,
   'processing_time_ms': 12.201757431030273,
   'requested_topn': 20,
   'coordinates': [{'x': 1032, 'y': 142},
    {'x': 1159, 'y': 145},
    {'x': 1156, 'y': 203},
    {'x': 1028, 'y': 200}],
   'vehicle_region': {'x': 910, 'y': 0, 

In [ ]:
len(dic_a)

Видим, что предсказаний меньше, чем картинок

## Получение данных для darknet из словаря dic_a

In [81]:
#Darknet требует создание txt-файла с содержанием <object-class> <x_center> <y_center> <width> <height>
#Рассчитаем эти значения, используя данные из словаря

In [64]:
dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates']

[{'x': 1263, 'y': 254},
 {'x': 1398, 'y': 258},
 {'x': 1396, 'y': 310},
 {'x': 1259, 'y': 306}]

In [66]:
# Переименуем координаты для удобства расчетов в формате xbr = x bottom right и т.д.
xbl = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][0]['x']
ybl = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][0]['y']
xbr = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][1]['x']
ybr = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][1]['y']
xtl = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][3]['x']
ytl = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][3]['y']
xtr = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][2]['x']
ytr = dic_a['407ETR_FC01_20140402_092945_00020.jpg'][0]['coordinates'][2]['y']

In [89]:
# Укажем размер картинок
image_w = 2048
image_h = 700

In [123]:
# посчитаем нужные для darknet значения
x_center_a = st.mean([xbl,xbr,xtl,xtr])/image_w
y_center_a = st.mean([ybl,ybr,ytl,ytr])/image_h
width_a = (st.mean([xbr,xtr])-st.mean([xbl,xtl]))/image_w
height_a = (st.mean([ytr,ytl])-st.mean([ybr,ybl]))/image_h
print(x_center_a, y_center_a, width_a, height_a)

0.64892578125 0.40285714285714286 0.06640625 0.07428571428571429


Если открыть картинку, выглядит похоже. По оси х и y центр примерно посередире. Размеры номера примерно соответствуют размерам относительно картинки.

## Словарь dic_b

In [130]:
with open('93.b.cache.pkl', 'rb') as pkl_file:
    dic_b = pickle.load(pkl_file)

dic_b

{'407ETR_FC01_20140402_092945_00020.jpg': '{"filename": "407ETR_FC01_20140402_092945_00020.jpg", "timestamp": "2021-07-07 11:15:06.990708", "camera_id": null, "results": [{"box": {"xmin": 1264, "ymin": 256, "xmax": 1390, "ymax": 302}, "plate": "bsyp875", "region": {"code": "", "score": 0}, "score": 0.89, "candidates": [{"score": 0.89, "plate": "bsyp875"}, {"score": 0.768, "plate": "8syp875"}], "dscore": 0.857, "vehicle": {"score": 0.274, "type": "Sedan", "box": {"xmin": 832, "ymin": 13, "xmax": 1806, "ymax": 364}}}], "usage": {"calls": 30321, "max_calls": 250000}, "processing_time": 331.276}',
 '407ETR_FC01_20140402_093341_00021.jpg': '{"filename": "407ETR_FC01_20140402_093341_00021.jpg", "timestamp": "2021-07-07 11:15:07.344965", "camera_id": null, "results": [{"box": {"xmin": 1024, "ymin": 137, "xmax": 1163, "ymax": 197}, "plate": "7464pn", "region": {"code": "ca", "score": 0.599}, "score": 0.904, "candidates": [{"score": 0.904, "plate": "7464pn"}], "dscore": 0.754, "vehicle": {"scor

In [125]:
# здесь уже все 10 тысяч записей, но данные все равно могут отсутствовать или быть не полными
len(dic_b)

10000

In [97]:
# Посмотрим на первый элемант словаря
dic_b['407ETR_FC01_20140402_092945_00020.jpg']

'{"filename": "407ETR_FC01_20140402_092945_00020.jpg", "timestamp": "2021-07-07 11:15:06.990708", "camera_id": null, "results": [{"box": {"xmin": 1264, "ymin": 256, "xmax": 1390, "ymax": 302}, "plate": "bsyp875", "region": {"code": "", "score": 0}, "score": 0.89, "candidates": [{"score": 0.89, "plate": "bsyp875"}, {"score": 0.768, "plate": "8syp875"}], "dscore": 0.857, "vehicle": {"score": 0.274, "type": "Sedan", "box": {"xmin": 832, "ymin": 13, "xmax": 1806, "ymax": 364}}}], "usage": {"calls": 30321, "max_calls": 250000}, "processing_time": 331.276}'

In [98]:
# Видим, что данные записаны в виде строки
type(dic_b['407ETR_FC01_20140402_092945_00020.jpg'])

str

## Получение данных для darknet из словаря dic_a

In [101]:
# Достанем координаты номера при помощи регулярных выражений. Ищем числа между "xmin": и запятой,
# Берем первое совпадение
xmin = int(re.compile(r'\"xmin\": (\d+),').findall(dic_b['407ETR_FC01_20140402_092945_00020.jpg'])[0])
ymin = int(re.compile(r'\"ymin\": (\d+),').findall(dic_b['407ETR_FC01_20140402_092945_00020.jpg'])[0])
xmax = int(re.compile(r'\"xmax\": (\d+),').findall(dic_b['407ETR_FC01_20140402_092945_00020.jpg'])[0])
ymax = int(re.compile(r'\"ymax\": (\d+)}').findall(dic_b['407ETR_FC01_20140402_092945_00020.jpg'])[0])
print(xmin,ymin,xmax,ymax)

1264 256 1390 302


In [126]:
# Теперь рассчитаем нужные для darknet значения, аналогично словарю dic_a

x_center_b = st.mean([xmin,xmax])/image_w
y_center_b = st.mean([ymin,ymax])/image_h
width_b = (xmax-xmin)/image_w
height_b = (ymax-ymin)/image_h
print(x_center_b, y_center_b, width_b, height_b)

0.64794921875 0.3985714285714286 0.0615234375 0.06571428571428571


# Создание текстовых файлов для каждой картинки

In [138]:
# Сделаем список из имен картинок, которые есть в папке
from os import listdir
from os.path import isfile, join
mypath = "93/"
image_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]

image_names[:5]

['407ETR_FC01_20140403_194712_06564.jpg',
 '407ETR_FC01_20140403_142942_05316.jpg',
 '407ETR_FC01_20140403_203404_06894.jpg',
 '407ETR_FC01_20140403_124731_04874.jpg',
 '407ETR_FC01_20140404_193536_01300.jpg']

In [ ]:
"""
План действий:
   Для каждого имени файла из списка:
        1) получим предсказания из словаря x
        2) получим предсказания из словаря y
        3) застакаем предсказания, взяв среднее
        4) создадим txt-файл с именем картинки и нужными для darknet данными внутри
"""

In [172]:
# Функция получения данных из словаря dic_a
def label_a(image_name):
# Переименуем координаты для удобства расчетов в формате xbr = x bottom right и т.д.
    xbl = dic_a[image_name][0]['coordinates'][0]['x']
    ybl = dic_a[image_name][0]['coordinates'][0]['y']
    xbr = dic_a[image_name][0]['coordinates'][1]['x']
    ybr = dic_a[image_name][0]['coordinates'][1]['y']
    xtl = dic_a[image_name][0]['coordinates'][3]['x']
    ytl = dic_a[image_name][0]['coordinates'][3]['y']
    xtr = dic_a[image_name][0]['coordinates'][2]['x']
    ytr = dic_a[image_name][0]['coordinates'][2]['y']
# посчитаем нужные для darknet значения
    x_center_a = st.mean([xbl,xbr,xtl,xtr])/image_w
    y_center_a = st.mean([ybl,ybr,ytl,ytr])/image_h
    width_a = (st.mean([xbr,xtr])-st.mean([xbl,xtl]))/image_w
    height_a = (st.mean([ytr,ytl])-st.mean([ybr,ybl]))/image_h
    
    return x_center_a, y_center_a, width_a, height_a

label_a('407ETR_FC01_20140402_092945_00020.jpg')

(0.64892578125, 0.40285714285714286, 0.06640625, 0.07428571428571429)

In [163]:
# Функция получения данных из словаря dic_b
def label_b(image_name):
# Достанем координаты номера при помощи регулярных выражений. Ищем числа между "xmin": и запятой,
# Берем первое совпадение
    xmin = int(re.compile(r'\"xmin\": (\d+),').findall(dic_b[image_name])[0])
    ymin = int(re.compile(r'\"ymin\": (\d+),').findall(dic_b[image_name])[0])
    xmax = int(re.compile(r'\"xmax\": (\d+),').findall(dic_b[image_name])[0])
    ymax = int(re.compile(r'\"ymax\": (\d+)}').findall(dic_b[image_name])[0])
# Теперь рассчитаем нужные для darknet значения, аналогично словарю dic_a
    x_center_b = st.mean([xmin,xmax])/image_w
    y_center_b = st.mean([ymin,ymax])/image_h
    width_b = (xmax-xmin)/image_w
    height_b = (ymax-ymin)/image_h
    
    return x_center_b, y_center_b, width_b, height_b

label_b('407ETR_FC01_20140402_092945_00020.jpg')

(0.64794921875, 0.3985714285714286, 0.0615234375, 0.06571428571428571)

In [219]:
for image_name in image_names:
    no_label_a = False
    no_label_b = False
    try:
        x_center_a, y_center_a, width_a, height_a = label_a(image_name)      
    except:
        no_label_a = True
        pass
    try:
        x_center_b, y_center_b, width_b, height_b = label_a(image_name)      
    except:
        no_label_a = True
        pass
# если не удалось достать информацию по картинке из словарей, выводим имя картинки, но таких случаев нет
# можно добавить обработку такого случая, например, удалить картинку, но я пока не стал этого делать
    if no_label_a == True and no_label_b == True:
        print(image)
# если удалось данные из обоих словарей, объединим результаты предсказаний, посчитав среднее значение
    if no_label_a == False and no_label_b == False:
        x_center = st.mean([x_center_a, x_center_b])
        y_center = st.mean([y_center_a, y_center_b])
        width = st.mean([width_a, width_b])
        height = st.mean([height_a, height_b])
# если данные только в первом словаре
    if no_label_a == False and no_label_b == True:
        x_center = x_center_a
        y_center = y_center_a
        width = width_a
        height = height_a      
# если данные только во втором словаре
    if no_label_a == True and no_label_b == False:
        x_center = x_center_b
        y_center = y_center_b
        width = width_b
        height = height_b
# переименуем image_name, удалив .jpg и добавив .txt
    image_name = image_name.split('.')[0]
    image_name += '.txt'
# запишем данные в файл
    file = open(image_name, "w") 
    file.write("0 %s %s %s %s" % (str(x_center),str(y_center),str(width),str(height)))
    file.close() 

# Создание файла train.txt

In [228]:
image_names_train = image_names[:8000]
file = open("data/train.txt", "w")
for image in image_names_train:
    file.write("data/obj/%s\n" % (image))
file.close() 

In [229]:
#Test.txt

In [230]:
image_names_train = image_names[9990:]
file = open("data/test.txt", "w")
for image in image_names_train:
    file.write("data/obj/%s\n" % (image))
file.close() 

# Darknet Train

In [ ]:
export PATH="$HOME/darknet:$PATH"

In [ ]:
darknet detector train ./data/obj.data ./data/yolo-obj.cfg ./data/yolov3-tiny.conv.11

# Darknet Test

In [ ]:
darknet detector test ./data/obj.data ./data/yolo-obj.cfg ./data/backup/yolo-obj_last.weights -dont_show -ext_output < data/test.txt > result.txt

# Итоговый скрипт на питоне

In [1]:
import pickle
import re
import statistics as st
import zipfile
image_w = 2048
image_h = 700
image_path = "data/obj/"

In [2]:
# распаковка картинок
with zipfile.ZipFile("93.zip", "r") as zip_ref:
    zip_ref.extractall(image_path)

In [3]:
# десереализация словарей
with open('93.a.cache.pkl', 'rb') as pkl_file:
    dic_a = pickle.load(pkl_file)
with open('93.b.cache.pkl', 'rb') as pkl_file:
    dic_b = pickle.load(pkl_file)

In [4]:
# создадим список из имен картинок, которые есть в папке
from os import listdir
from os.path import isfile, join
image_names = [f for f in listdir(image_path) if isfile(join(image_path, f))]

In [5]:
# создадим txt-файл с перечислением имён 80% картинок для тренировочной выборки
image_names_train = image_names[:8000]
file = open("data/train.txt", "w")
for image in image_names_train:
    file.write("data/obj/%s\n" % (image))
file.close()

In [6]:
# создадим txt-файл с перечислением имён 10 картинок для теста
image_names_train = image_names[9990:]
file = open("data/test.txt", "w")
for image in image_names_train:
    file.write("data/obj/%s\n" % (image))
file.close() 

In [7]:
# Функция получения данных из словаря dic_a
def label_a(image_name):
# Переименуем координаты для удобства расчетов в формате xbr = x bottom right и т.д.
    xbl = dic_a[image_name][0]['coordinates'][0]['x']
    ybl = dic_a[image_name][0]['coordinates'][0]['y']
    xbr = dic_a[image_name][0]['coordinates'][1]['x']
    ybr = dic_a[image_name][0]['coordinates'][1]['y']
    xtl = dic_a[image_name][0]['coordinates'][3]['x']
    ytl = dic_a[image_name][0]['coordinates'][3]['y']
    xtr = dic_a[image_name][0]['coordinates'][2]['x']
    ytr = dic_a[image_name][0]['coordinates'][2]['y']
# посчитаем нужные для darknet значения
    x_center_a = st.mean([xbl,xbr,xtl,xtr])/image_w
    y_center_a = st.mean([ybl,ybr,ytl,ytr])/image_h
    width_a = (st.mean([xbr,xtr])-st.mean([xbl,xtl]))/image_w
    height_a = (st.mean([ytr,ytl])-st.mean([ybr,ybl]))/image_h
    
    return x_center_a, y_center_a, width_a, height_a

In [8]:
# Функция получения данных из словаря dic_b
def label_b(image_name):
# Достанем координаты номера при помощи регулярных выражений. Ищем числа между "xmin": и запятой,
# Берем первое совпадение
    xmin = int(re.compile(r'\"xmin\": (\d+),').findall(dic_b[image_name])[0])
    ymin = int(re.compile(r'\"ymin\": (\d+),').findall(dic_b[image_name])[0])
    xmax = int(re.compile(r'\"xmax\": (\d+),').findall(dic_b[image_name])[0])
    ymax = int(re.compile(r'\"ymax\": (\d+)}').findall(dic_b[image_name])[0])
# Теперь рассчитаем нужные для darknet значения, аналогично словарю dic_a
    x_center_b = st.mean([xmin,xmax])/image_w
    y_center_b = st.mean([ymin,ymax])/image_h
    width_b = (xmax-xmin)/image_w
    height_b = (ymax-ymin)/image_h
    
    return x_center_b, y_center_b, width_b, height_b

In [9]:
for image_name in image_names:
    no_label_a = False
    no_label_b = False
    try:
        x_center_a, y_center_a, width_a, height_a = label_a(image_name)      
    except:
        no_label_a = True
        pass
    try:
        x_center_b, y_center_b, width_b, height_b = label_a(image_name)      
    except:
        no_label_a = True
        pass
# если не удалось достать информацию по картинке из словарей, выводим имя картинки, но таких случаев нет
# можно добавить обработку такого случая, например, удалить картинку, но я пока не стал этого делать
    if no_label_a == True and no_label_b == True:
        print(image)
# если удалось данные из обоих словарей, объединим результаты предсказаний, посчитав среднее значение
    if no_label_a == False and no_label_b == False:
        x_center = st.mean([x_center_a, x_center_b])
        y_center = st.mean([y_center_a, y_center_b])
        width = st.mean([width_a, width_b])
        height = st.mean([height_a, height_b])
# если данные только в первом словаре
    if no_label_a == False and no_label_b == True:
        x_center = x_center_a
        y_center = y_center_a
        width = width_a
        height = height_a      
# если данные только во втором словаре
    if no_label_a == True and no_label_b == False:
        x_center = x_center_b
        y_center = y_center_b
        width = width_b
        height = height_b
# переименуем image_name, удалив .jpg и добавив .txt
    image_name = image_name.split('.')[0]
    image_name += '.txt'
    txt_path = image_path+image_name
# запишем данные в файл
    file = open(txt_path, "w") 
    file.write("0 %s %s %s %s" % (str(x_center),str(y_center),str(width),str(height)))
    file.close() 